# Cuentos Generator

## Get the Data

In [6]:
import helper

data_dir = './data/cuentos.txt'
text = helper.load_data(data_dir)

In [7]:
# Ignore \ufeff
text = text[1:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [9]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 21485
Number of scenes: 102
Average number of sentences in each scene: 21.970588235294116
Number of lines: 2343
Average number of words in each line: 43.56380708493385

The sentences 0 to 10:
Al pie de la Biblia abierta –donde estaba señalado en rojo el versículo que lo explicaría todo– alineó las cartas: a su mujer, al juez, a los amigos. Después bebió el veneno y se acostó.
Nada. A la hora se levantó y miró el frasco. Sí, era el veneno.
¡Estaba tan seguro! Recargó la dosis y bebió otro vaso. Se acostó de nuevo. Otra hora. No moría. Entonces disparó su revólver contra la sien. ¿Qué broma era ésa? Alguien -¿pero quién, cuándo?- alguien le había cambiado el veneno por agua, las balas por cartuchos de fogueo. Disparó contra la sien las otras cuatro balas. Inútil. Cerró la Biblia, recogió las cartas y salió del cuarto en momentos en que el dueño del hotel, mucamos y curiosos acudían alarmados por el estruendo de los cinco estampidos.
Al ll

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [13]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    vocab = []
    for word in text:
        if not (word in vocab):
            vocab.append(word)
    n_words = len(vocab)
    return {vocab[i]:i for i in range(n_words)}, {i:vocab[i] for i in range(n_words)}

create_lookup_tables(['a', 'b', 'a'])

({'a': 0, 'b': 1}, {0: 'a', 1: 'b'})

In [14]:
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [15]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    return {
        '.' : '||Period||', 
        ',' : '||Comma||', 
        '"' : '||QuotationMark||', 
        ';' : '||Semicolon||',
        '!' : '||ExclamationMark||',
        '?' : '||QuestionMark||',
        '(' : '||LeftParentheses||',
        ')' : '||RightParentheses||',
        '--': '||Dash||',
        '\n': '||Return||'}

tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [16]:
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [17]:
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [18]:
len(int_text)

118431

In [19]:
int_text[:20]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 8, 5, 11, 12, 13, 14, 15, 16, 17]

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [20]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0


/Users/bartekskorulski/Envs/tf1.0/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: No GPU found. Please use a GPU to train your neural network.
  # This is added back by InteractiveShellApp.init_path()


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [21]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    target = tf.placeholder(tf.int32, [None, None])
    learning_rate = tf.placeholder(tf.float32)
    return inputs, target, learning_rate

tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [22]:
LSTM_LAYERS = 1

def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    stacked_lstm_cell = tf.contrib.rnn.MultiRNNCell([lstm] * LSTM_LAYERS)
    initial_state = tf.identity(stacked_lstm_cell.zero_state(batch_size, tf.float32), name = 'initial_state')
    return stacked_lstm_cell, initial_state


tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [23]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed

tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [24]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, state = tf.nn.dynamic_rnn(cell, inputs=inputs, dtype=tf.float32)
    final_state = tf.identity(state, name="final_state")
    return outputs, final_state

tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [25]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    embed = get_embed(input_data, vocab_size, embed_dim) 
    outputs, final_state = build_rnn(cell, embed)
    output_rnn = outputs[-1]
    fc_out = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn = None)
    return fc_out, final_state

tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [26]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    n_words = len(int_text)
    batch_len = batch_size * seq_length
    n_batches = n_words//batch_len
    x = int_text[ : n_batches * batch_len]
    y = x[1:] + [x[0]]
    x = np.reshape(x, (batch_size, -1))
    y = np.reshape(y, (batch_size, -1))
    batches = []
    for i in range(n_batches):
        batches.append([
            x[:, i*seq_length:(i+1)*seq_length],
            y[:, i*seq_length:(i+1)*seq_length]
        ])
    return np.array(batches)
           
get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26], 3, 2)

array([[[[ 1,  2],
         [ 9, 10],
         [17, 18]],

        [[ 2,  3],
         [10, 11],
         [18, 19]]],


       [[[ 3,  4],
         [11, 12],
         [19, 20]],

        [[ 4,  5],
         [12, 13],
         [20, 21]]],


       [[[ 5,  6],
         [13, 14],
         [21, 22]],

        [[ 6,  7],
         [14, 15],
         [22, 23]]],


       [[[ 7,  8],
         [15, 16],
         [23, 24]],

        [[ 8,  9],
         [16, 17],
         [24,  1]]]])

In [27]:
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [59]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 32
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 200
# Sequence Length
seq_length = 128
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 256//batch_size

save_dir = './save'

In [60]:
show_every_n_batches

8

### Build the Graph
Build the graph using the neural network you implemented.

In [61]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [ ]:
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/28   train_loss = 9.654
Epoch   0 Batch    8/28   train_loss = 7.463
Epoch   0 Batch   16/28   train_loss = 7.109
Epoch   0 Batch   24/28   train_loss = 6.719
Epoch   1 Batch    4/28   train_loss = 6.339
Epoch   1 Batch   12/28   train_loss = 6.164
Epoch   1 Batch   20/28   train_loss = 5.975
Epoch   2 Batch    0/28   train_loss = 5.669
Epoch   2 Batch    8/28   train_loss = 5.529
Epoch   2 Batch   16/28   train_loss = 5.610
Epoch   2 Batch   24/28   train_loss = 5.484
Epoch   3 Batch    4/28   train_loss = 5.340
Epoch   3 Batch   12/28   train_loss = 5.268
Epoch   3 Batch   20/28   train_loss = 5.264
Epoch   4 Batch    0/28   train_loss = 5.108
Epoch   4 Batch    8/28   train_loss = 4.974
Epoch   4 Batch   16/28   train_loss = 5.001
Epoch   4 Batch   24/28   train_loss = 4.890
Epoch   5 Batch    4/28   train_loss = 4.812
Epoch   5 Batch   12/28   train_loss = 4.756
Epoch   5 Batch   20/28   train_loss = 4.618
Epoch   6 Batch    0/28   train_loss = 4.561
Epoch   6 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [53]:
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [54]:
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [55]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [56]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    i = np.random.choice(range(len(probabilities)), p=probabilities)
    return int_to_vocab[i]

tests.test_pick_word(pick_word)

Tests Passed


## Generate Story
This will generate the story.  Set `gen_length` to the length of story you want to generate.

In [58]:
gen_length = 100


gen_sentences = ["tomó", "el", "cuchillo", "de", "la",  "cocina"]
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

tomó el cuchillo de la cocina y le ofreciera interés y la royal, amable–. dimos mi permiso. se abrió pareció que no era una finalidad esperanza de curar a su sueño. pero a eso no la corte con mi silencioso acompañante y anduvo rugiendo por el cielo, recortado de ellos o conradín: era personalmente de respeto. ya no dejaba, mientras este extremo, mentando el cuchillo de un mal humor inaguantable, y al oído con él no fuera por ya? -preguntó hathaway, convertido sobre la primera tendencia del cielo raso se pone en que no le
